In [2]:
import mysql.connector
import pandas as pd
import os
from dotenv import load_dotenv

# 환경 변수 로드
load_dotenv()
db_password = os.getenv('password')

# --- 1. MySQL 연결 및 데이터 추출 함수 ---
# (이 부분은 이전 코드와 동일합니다.)
def get_db_connection():
    """DB 연결 객체를 반환합니다."""
    try:
        conn = mysql.connector.connect(
            host="127.0.0.1",
            user="root",
            password=db_password,
            database="new_schema"
        )
        return conn
    except mysql.connector.Error as err:
        print(f"Database Connection Error: {err}")
        return None

def load_data_to_df(conn, table_name, query="SELECT * FROM {table_name}"):
    """테이블 이름과 쿼리를 받아 Pandas DataFrame을 반환합니다."""
    
    final_query = query.format(table_name=table_name)
    
    try:
        # SQL 쿼리를 DataFrame으로 바로 읽습니다. (전체 데이터를 로드할 경우 시간이 소요될 수 있습니다.)
        df = pd.read_sql(final_query, conn)
        print(f"✅ {table_name} 데이터 로드 성공 (총 {len(df)}개 행)")
        return df
    except Exception as e:
        print(f"❌ {table_name} 데이터 로드 실패: {e}")
        return pd.DataFrame() 

# --- 2. 데이터 정리 및 통합 메인 로직 ---
def prepare_ui_data():
    conn = get_db_connection()
    if conn is None:
        return {} 

    data_frames = {}
    
    # a) 버스 노선 데이터 (지도 활용)
    data_frames['bus'] = load_data_to_df(conn, 'bus_routes2', 
                                          query="SELECT 정류장명, 경도, 위도 FROM {table_name}")
    
    # b) 사업체 위치 데이터 (지도 활용)
    data_frames['business_coords'] = load_data_to_df(conn, 'chuncheon_businesses_with_coords',
                                          query="SELECT 업종, 업소명, 위도, 경도 FROM {table_name}")
    
    # c) 상권 기본 정보 (대시보드 KPI 및 JOIN 기준)
    data_frames['area_info'] = load_data_to_df(conn, 'commercial_area',
                                               query="""SELECT 상권업종대분류명, 시군구명, 경도, 위도 FROM {table_name}""")
    
    # d) 상권별 사업체 상세 정보 (JOIN 대상)
    data_frames['sanggwon_biz'] = load_data_to_df(conn, 'sanggwon_businesses',
                                               query="SELECT 상호명, 업종명, 주소 FROM {table_name}")
    
    # ⭐️ e) 강원도 사업체 상세 목록 (비교 분석 및 목록 표시용)
    data_frames['kangwon_biz'] = load_data_to_df(conn, 'kangwondo_businesses',
                                               query="SELECT 업종, 업소명, 주소, 연락처 FROM {table_name}")
    
    # --- 데이터 정리 및 컬럼명 통일 ---
    
    # 1. 버스 데이터 컬럼 통일
    if not data_frames['bus'].empty:
        data_frames['bus'].columns = ['station_name', 'lon', 'lat']
        data_frames['bus'] = data_frames['bus'].dropna(subset=['lon', 'lat'])

    # 2. 사업체 위치 컬럼 통일
    if not data_frames['business_coords'].empty:
        data_frames['business_coords'].columns = ['type', 'name', 'lat', 'lon']
        data_frames['business_coords'] = data_frames['business_coords'].dropna(subset=['lat', 'lon'])
        
    # 3. 강원도 사업체 컬럼 통일 (지도 데이터와 구조가 다르므로 분리 유지)
    if not data_frames['kangwon_biz'].empty:
        data_frames['kangwon_biz'].columns = ['type', 'name', 'address', 'contact']
        
    # (나머지 area_info 및 sanggwon_biz 정리 로직 생략, 기존과 동일)
        
    conn.close()
    
    print("\n--- 데이터 정리 완료 ---")
    
    # 최종적으로 UI에 전달할 핵심 데이터셋 딕셔너리 반환
    return {
        "bus_stations": data_frames['bus'].to_dict('records'),
        "businesses": data_frames['business_coords'].to_dict('records'),
        "all_businesses_list": data_frames['kangwon_biz'].to_dict('records'), # 새로운 데이터셋
        "area_info": data_frames['area_info'].to_dict('records'),
        "sanggwon_details": data_frames['sanggwon_biz'].to_dict('records')
    }

# --- 실행 ---
if __name__ == "__main__":
    prepared_data = prepare_ui_data()
    
    print(f"\n준비된 최종 데이터셋 키: {list(prepared_data.keys())}")
    print("\n--- 강원도 사업체 목록 데이터 샘플 (활용 예시) ---")
    print(prepared_data['all_businesses_list'][:2])
    print("\n이제 이 데이터를 활용하여 UI를 구축할 수 있습니다.")

/var/folders/n7/7zp7j9413xsgfgqknxqw7h880000gn/T/ipykernel_11751/1591753474.py:33: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(final_query, conn)


✅ bus_routes2 데이터 로드 성공 (총 31560개 행)
✅ chuncheon_businesses_with_coords 데이터 로드 성공 (총 34개 행)
✅ commercial_area 데이터 로드 성공 (총 17497개 행)
✅ sanggwon_businesses 데이터 로드 성공 (총 107631개 행)
✅ kangwondo_businesses 데이터 로드 성공 (총 356개 행)

--- 데이터 정리 완료 ---

준비된 최종 데이터셋 키: ['bus_stations', 'businesses', 'all_businesses_list', 'area_info', 'sanggwon_details']

--- 강원도 사업체 목록 데이터 샘플 (활용 예시) ---
[{'type': '한식', 'name': '청재', 'address': '강원도 춘천시 서면 박사로 927', 'contact': '033-255-2867'}, {'type': '한식', 'name': '두리감자탕', 'address': '강원도 춘천시 후평동 748-4', 'contact': '033-244-2847'}]

이제 이 데이터를 활용하여 UI를 구축할 수 있습니다.
